

# How to $\mathcal{P} \mathfrak{lon} \mathcal{K}$ 

*Martín Ochoa, zkSecurity*


Welcome to the PLONK tutorial! There are a couple alternative ways you can use this tutorial:

* You can download the [Jupyter Notebook](code/notebook.zip) version and run it locally on a [Sage kernel](https://doc.sagemath.org/html/en/installation/launching.html) or on the cloud on [Cocalc](https://cocalc.com/).
* You can follow this page and implement the solutions locally in Sage or some other programming language. We aim at giving deterministic test cases to guide you.

**Note**: It is highly recommended you use Sage solve the exercises below in order to take advantage of its libraries, and of our implementation of the algebra needed for pairings over the bn254 curve.

If you have any feedback or comments do let us know at *martin$@$zksecurity$.$xyz*.




## 1.Introduction

In this notebook we illustrate the [PLONK](https://eprint.iacr.org/2019/953.pdf) zero-knowledge-proof protocol on a simple example that we will progressively make more complete. The goal is to understand the mathematical inner workings of the protocol, down to the level of cryptographic pairings and commitment schemes. The PLONK protocol has various phases each of which involves technical steps in order to achieve certain protocol subgoals. 

## 2.A first example

Consider the following scenario: You have a low resource device $V$ (a *validator*) that can delegate computations to a *processor* $P$, which has more computing capabilities. Sometimes the processor $P$ makes mistakes in the computations, so $V$ wants to do some sanity checks on $V$ to make sure the final value of the computation is correct. 


#### Fibonacci Example

For example, let's assume that $V$ asks $P$ to compute the $4$-th Fibonacci number squared, that is $F_{4}^2$. Recall that the Fibonacci sequence is defined by:


$$F_n = F_{n-1} + F_{n-2}$$

with the base cases $F_0 = 0$ and $F_1 = 1$.

Note that this computation can be expressed as a circuit as illustrated in Figure 1.

<center><img src="img_td/fibonacci_squared_circuit.png" width="600"></center>

The goal of this example will be to design a first simple protocol that allows us to validate whether this circuit has been correctly evaluated by the processor $P$ without having to recompute every single intermediate state (wire value) of the circuit. In other words the question is: can we devise a protocol that can (efficiently) guarantee that the $F_{4}$ value of the sequence has been correctly computed and then squared? Are the base cases $F_0$ and $F_1$ correct, and have all intermediate computations of the circuit correctly performed? 

To achieve this, let us start by picturing the circuit and all intermediate computations as a table. Consider the following picture.


<center><img src="img_td/fibonacci_s_table.png" width="350"></center>

Note that the condition that $F_4^2$ is correctly computed according to the Fibonacci recursive definition is equivalent to the following conditions:

* The left and right inputs to the first gate are $0$ and $1$ respectively.
* For all gates 1 to 3 the output is computed as the sum of the left and right inputs
* For gates 2 and 3 the left input of gate $k$ is equal to the right input to gate $k-1$ and the right input of gate $k$ is the output of gate $k-1$.
* For gate 4, the left and right inputs are the output of gate 3, and the output is the multiplication of the inputs.

We can visualize this as follows:

<center><img src="img_td/fibonacci_s_table2.png" width="350"></center>

Or mathematically, consider the set of indexes $I=\{1,2,3,4\}$ and denote the left inputs as a function $$LI: I \to \mathbb{N}$$ and similarly the right inputs $RI$ and the outputs $O$. Then we could say:


$$\begin{aligned}
&LI(1)=0,\quad RI(1)=1,\\
&\forall i \in I\setminus \{4\} : LI(i)+RI(i) = O(i),\\ 
& \forall i \in I\setminus \{3,4\}:  \bigl(LI(i+1) = RI(i)\bigr)\;\wedge\;\bigl(RI(i+1)=O(i)\bigr),\\
&LI(4)=RI(4) = O(3),\\   
&LI(4)*RI(4) = O(4),\\
&\implies\quad O(4) = F_4^2.
\end{aligned}$$

So if the processor $P$ can convince the validator $V$ that the above defined constraints hold for the table representation of the circuit computation, then the validator will be convinced that $F_4^2$ was computed correctly. Let us now explore a way to do so by leveraging on polynomials and so-called zero-equality tests.

### Interpolation and Zero-equality tests

A trivial way for the verifier to check if the constraints on the circuit hold would be to get copies of $LI,RI$ and $O$ and check them. But this would defeat the purpose of delegating the computation, because the amount of work that the prover has to do is equivalent to recomputing $F_4^2$ on his own. To make this more efficient, consider the following encoding of the vectors $LI,RI$ and $O$: suppose there was an easy way to transform those vectors into polynomials $a(x),b(x),c(x)$ such that, for every index $i \in I$, $a(i) = LI(i)$, $b(i)= RI(i)$ and $c(i)=O(i)$. 

If this was possible, then the processor $P$ could hand in $a(x),b(x)$ and $c(x)$ to the validator, who can compute:

$$ t(x) = a(x) + b(x) - c(x)$$

This polynomial has the property that for all indexes $i \in I\setminus{4}$, $t(i) = 0$. 

Additionally, the validator needs to check that:

$$ t'(x) = a(x)*b(x) - c(x)$$

is zero for $x=4$. 

#### Selectors
In order to simplify those checks, PLONK introduces the so-called selector columns (and their respective polynomials as follows). Consider three boolean vectors $SL, SR, SM$ such that $SL$ and $SR$ are set if a given index contains an addition gate, and zero otherwise. Likewise, $SM$ will be set whenever there is a multiplication gate. Their polynomial version satisfy $qL(i) = SL(i), qR(i)=SR(i), qM(i)=SM(i)$. Then we could compress checks for both addition and multiplication gates into one:

$$t(x)= qL(x) \cdot a(x) + qR(x) \cdot b(x) + qM(x) \cdot (a(x) \cdot b(x)) - c(x) $$

This polynomial should now be 0 for all $i \in I$.


#### Wiring constraints

Moreover, we could also encode the missing constraints as follows:

$$ f_1(x) = a(x+1) - b(x)$$
$$f_2(x) = b(x+1) - c(x)$$


These two should also be zero at all but the last two indexes, $i \in I \setminus{\{3,4\}}$. Last we need to check:

$$f_3(x) = b(x) - c(x-1)$$

$$f_4(x) = a(x) - b(x)$$

which should hold for $x=4$.


So by encoding the vectors $L,R,O$ into polynomials $a(x),b(x),c(x)$, a validator $V$ could in principle check all that is needed to convince themselves that $F_4^2 = c(4)$ was computed correctly.

**Let us begin implementing these ideas** before building our (still preliminary) validation protocol. Although mathematically the Fibonacci function is defined over the naturals $\mathbb{N}$, when implementing zero-knowledge proofs we crucially do all arithmetic in a finite field $\mathbb{F}_p$, meaning values are taken modulo a prime $p$. There are various reasons why this is done. First, many cryptographic constructions rely on the difficulty of certain discrete problems over finite fields. Also, working on a field ensures every nonzero element has a multiplicative inverse, which will be useful in the following to reason about operations over polynomials. Last, arithmetic modulo $p$ is straightforward to implement and typically efficient.

**Exercise 1** Represent the vectors $LI,RI,O,SL,SR,SM$ as polynomials $a,b,c$ over the finite field defined by $$p = 21888242871839275222246405745257275088548364400416034343698204186575808495617.$$ This prime number is the order of the bn254 elliptic curve, which will be useful in the following. To interpolate the polynomials use Lagrange interpolation as follows. Given a set of points $X = \{x_1,x_2,\dots,x_n\}$ and data $(x_i,y_i)$ for each $i \in I$, 
the interpolating polynomial $f(x)$:

$$
f(x) 
= 
\sum_{j \in I} 
\Bigl(
  y_j 
  \prod_{\substack{i \in I \\ i \neq j}}
  \frac{x - x_i}{x_j - x_i}
\Bigr)
\;\in\;\mathbb{F}_p[x].
$$

In [ ]:
def interpolate(I, Y):
    #Solve here!
    return f

p = 21888242871839275222246405745257275088548364400416034343698204186575808495617
F = GF(p)
R.<x> = PolynomialRing(F, 'x')   

LI = 
RI = 
O = 
I = 

SL=
SR=
SM=


qL = 
qR = 
qM = 


a =
b = 
c =

If computed correctly you should be able to check the following properties on the polynomials:

* They are all polynomials of degree 3 over $\mathbb{F}_p$
* When evaluated over the indexes $I$ you get the expected values as defined by $L,R,O$.
* When defining $t = a(x) + b(x) - c(x)$ you get 0 over the indexes $I$.

For example, the first polynomial is:

$$a(x) = 10944121435919637611123202872628637544274182200208017171849102093287904247809 x^{3} + 10944121435919637611123202872628637544274182200208017171849102093287904247805 x^{2} + 8 x + 21888242871839275222246405745257275088548364400416034343698204186575808495612$$

In [2]:
all = True
for i in I:
    all = all and (a(i) == LI[i-1])
print("Is a(x) correctly interpolated?", all)
all = True
for i in I:
    all = all and (b(i) == RI[i-1])
print("Is b(x) correctly interpolated?", all)
all = True
for i in I:
    all = all and (c(i) == O[i-1])
print("Is c(x) correctly interpolated?", all)
all = True
t = qM*a*b + qL*a+qR*b-c
for i in I:
    all = all and (t(i) == 0)
print("Is  t =qM*a*b + qL*a+qR*b-c zero over all indexes?", all)


Is a(x) correctly interpolated? True
Is b(x) correctly interpolated? True
Is c(x) correctly interpolated? True
Is  t =qM*a*b + qL*a+qR*b-c zero over all indexes? True


At this point we can also check the recursive constraints.

In [3]:
f1 = a(x+1)-b(x)
all = True
for i in I[:2]:
    all = all and ( f1(i) == 0)
print("Is f1(x) = a(x+1)- b(x) zero over all indexes minus the last?", all)

f2 = b(x+1)-c(x)
all = True
for i in I[:2]:
    all = all and ( f1(i) == 0)
print("Is f2(x) = b(x+1)- c(x) zero over all indexes minus the last?", all)

Is f1(x) = a(x+1)- b(x) zero over all indexes minus the last? True
Is f2(x) = b(x+1)- c(x) zero over all indexes minus the last? True


### Alternative zero-equality testing

Now, remember we want to build a protocol that is more efficient than recomputing the original function. In the checks above, we were looping through the values of $I$ to check for zero equality. We can take advantage of the algebraic properties of the interpolated polynomials to do this in a different way, which will lead to a more efficient algorithm for arbitrary circuits.


In fact, since we are working on a field, we know that if $t(x)=0$ for every index $i \in I$, then the polynomial $(x-i)$ divides $t(x)$, and therefore the polynomial $Z(x) = \prod_{i \in I} (x-i)$ divides $t(x)$. Equivalently, there must exist a quotient polynomial $Q(x) \in \mathbb{F}[x]$ such that:

$$ t(x) = Q(x) \cdot Z(x)$$

Checking this relation for the right "vanishing" polynomial Z replaces the need for looping over all indexes.

**Exercise 2**:  Compute $Z(x)$ and quotient $Q(x)$ for $t(x)$. Also compute $Z'(x)$ for $I' = I \setminus{\{3,4\}}$ and the corresponding $Q_1(x)$ and $Q_2(x)$ for the recursive constraint polynomials $f_1$ and $f_2$.

In [ ]:
#Solve here!

Z = 
Q = 
Z1 = 
Q1 = 
Q2 = 

If computed correctly you should get:

$$Q(x) = 304003373219989933642311190906351042896505061116889365884697280369108451328 x^{5} + 18240202393199396018538671454381062573790303667013361953081836822146507079681 x^{4} + 15504172034219486615757870736223903187721758116961357660119561298824531017728 x^{3} + 12160134928799597345692447636254041715860202444675574635387891214764338053122 x^{2} + 7296080957279758407415468581752425029516121466805344781232734728858602831871 x + 7296080957279758407415468581752425029516121466805344781232734728858602831873
$$

We can also do the sanity check of recovering $t$ from $Q$ and $Z$.

In [5]:
print("Is t(x) = Q(x) * Z(x)?", t == Q*Z)
show("Q=",Q)
print("Is f1(x) = Q1(x) * Z1(x)?", f1 == Q1*Z1)
show("Q1=",Q1)
print("Is f2(x) = Q2(x) * Z1(x)?", f2 == Q2*Z1)
show("Q2=",Q2)

Is t(x) = Q(x) * Z(x)? True


'Q=' 304003373219989933642311190906351042896505061116889365884697280369108451328*x^5 + 18240202393199396018538671454381062573790303667013361953081836822146507079681*x^4 + 15504172034219486615757870736223903187721758116961357660119561298824531017728*x^3 + 12160134928799597345692447636254041715860202444675574635387891214764338053122*x^2 + 7296080957279758407415468581752425029516121466805344781232734728858602831871*x + 7296080957279758407415468581752425029516121466805344781232734728858602831873

Is f1(x) = Q1(x) * Z1(x)? True


'Q1=' 7296080957279758407415468581752425029516121466805344781232734728858602831873*x + 10944121435919637611123202872628637544274182200208017171849102093287904247807

Is f2(x) = Q2(x) * Z1(x)? True


'Q2=' 21888242871839275222246405745257275088548364400416034343698204186575808495616*x + 3

#### Recap and First Protocol

We now have the tools to define a **preliminary** validation protocol for computing 
the squared 4th Fibonacci number. Here's an outline of each party's steps:

##### 1. Setup

1. *(Both)* Fix a working field $\mathbb{F}_p$ and note that we will interpolate polynomials of degree $\le 3$.
2. *(Both)* Compute selector polynomials $qL,qR,qM$ corresponding to this circuit.



##### 2. Prover Computation

1. *(Prover $P$)* Computes the 4th Fibonacci number squared $F_4^2$ by forming a table of intermediate values, 
   then interpolates them as **3th-degree polynomials**:
   $$
   a(x), \quad b(x), \quad c(x).
   $$

   as well as selectors $qL(x), \quad qR(x), \quad qM(x)$.
3. *(Prover $P$)* Computes 
   $$
   t(x)= qL(x) \cdot a(x) + qR(x) \cdot b(x) + qM(x) \cdot (a(x) \cdot b(x)) - c(x) 
   $$
   as well as the **quotient polynomial** $Q(x)$ satisfying
   $$
   t(x) \;=\; Q(x)\,\cdot Z(x).
   $$
4. *(Prover $P$)* Similarly constructs 
   $$
   f_1(x) \;=\; a(x+1) \;-\; b(x), 
   \quad
   f_2(x) \;=\; b(x+1) \;-\; c(x),
   $$
   along with their quotient polynomials 
   $$
   Q_1(x), \quad Q_2(x)
   $$
   using $Z_1(x)$ for the domain $\{1,2\}$.


##### 3. Prover → Verifier Communication

1. *(Prover $P$)* Sends the polynomials 
   $$
   a(x), \; b(x), \; c(x), \; Q(x), \; Q_1(x), \; Q_2(x)
   $$
   to the Validator $V$.


##### 4. Verifier Checks

1. *(Verifier $V$)* Checks the base conditions: 
   $$
   a(1) \;=\; 0, 
   \quad
   b(1) \;=\; 1.
   $$
2. *(Verifier $V$)* Computes $t(x) = qL(x) \cdot a(x) + qR(x) \cdot b(x) + qM(x) \cdot (a(x) \cdot b(x)) - c(x)$ and verifies 
   $$
   t(x) \;=\; Q(x)\,\cdot Z(x).
   $$
   That is, $t(x)$ is indeed divisible by $Z(x)$.
3. *(Verifier $V$)* Checks the recursive constraints:
   $$
   f_1(x) \;=\; Q_1(x)\,\cdot Z_1(x),
   \quad
   f_2(x) \;=\; Q_2(x)\,\cdot Z_1(x).
   $$
4. *(Verifier $V$)* Checks the square gate constraints:
   $$
   f_3(4) \;=\; c(3)-b(4)=0 ,
   \quad
   f_4(4) \;=\; a(4)-b(4)= 0.
   $$
   
5. *(Verifier $V$)* If all checks pass, the Verifier concludes 
   $$
   c(4) \; \text{is the correct value of } F_4^2.
   $$


<center>
  <img src="img_td/first_protocol_s.png" width="500">
</center>

This protocol is correct but seems quite inefficient:
instead of doing only 3 additions and one multiplication, we end up performing polynomial multiplications of polynomials of degree 3 over
a very large prime field $\mathbb{F}_p$. We will now consider a more efficient version.

## 3.Towards a more efficient version

We have considered two ways in which the validator $V$ can check whether zero-equality constraints are met: either evaluating the constraint for reach gate of the circuit (i.e. evaluating the polynomial $t(x)$ encoding the sum constraint on each index), or computing the polynomial multiplication between the quotient polynomial $Q(x)$ and the vanishing polynomial $Z(x)$ and check equality against the polynomial encoding the constraints (i.e. $t(x)=Q(x) \cdot Z(x)$). Among the reasons why the second alternative is advantageous is that we can perform the equality check very efficient with a probabilistic algorithm at the price of (very rarely) making some mistakes.

Consider the following algorithm:

* The validator has $t(x)$, $Q(x)$ and $Z(x)$ and wants to know whether $t(x) = Q(x) \cdot Z(x)$.
* They sample a random value $\gamma \in \mathbb{F}_p$.
* They check whether $t(\gamma) = Q(\gamma) \cdot Z(\gamma)$.

Why does this work? Certainly this check is a *necessary* condition, that is, if the polynomial equality holds, then it also holds when evaluated on a single point. But is it sufficient? In other words, can it be that $t(\gamma) = Q(\gamma) \cdot Z(\gamma)$ holds but $t(x) \neq Q(x) \cdot Z(x)$? It turns out that yes, that can unfortunately happen, but we can count on how many points this may at most happen, and therefore compute a probability for that event which allows us to reason on how "risky" our protocol is.

**Counting errors** Consider two polynomials $f(x), g(x) \in \mathbb{F}_p[x]$ of degree at most $d$, such that $f(x) \neq g(x)$. Consider the set of points $S \subseteq \mathbb{F}_p$ such that $\forall s \in S: f(s) = g(s)$. This set of points $S$ is exactly the set of points for which $f(x) - g(x)=0$. By assumption $f(x) \neq g(x)$ so $h(x) = f(x)-g(x) \in \mathbb{F}_p[x]$ is a non trivial polynomial of degree at most $d$. This polynomial $h(x)$ has at most $d$ roots, so the size of $S$ is bounded $|S| \leq d$. Given that we are sampling a value $\gamma$ uniformly at random from $\mathbb{F}_p$, the probability of selecting a point in $S$, in case $f(x) \neq g(x)$, is $\frac{|S|}{p} \leq \frac{d}{p}$. This is known as the [Schwartz-Zippel Lemma](https://en.wikipedia.org/wiki/Schwartz%E2%80%93Zippel_lemma) (which is more generally stated in a multi-variate polynomial setting).

In our circuit example, given that the polynomials involved are of degree 3, we get an extremely low probability of an incorrect equality check of at most $\frac{3}{p} \approx \frac{1}{2^{252}}$. 

**Exercise 3** Compute the equality checks for the gate and recursive constraints with the following random values: $\gamma_1 = 42$, $\gamma_2 = 74102$ and $\gamma_3 = 987654321987654321$. You should obtain:

$$\begin{aligned}
t(\gamma_1)
  &= Q(\gamma_1) \cdot Z(\gamma_1)
   = 21888242871839275222246405745257275088548364400416034343698204183303103172977\\
f_1(\gamma_2)
  &= Q_1(\gamma_2) \cdot Z_1(\gamma_2)
   = 271248759392650\\
f_2(\gamma_3)
  &= Q_2(\gamma_3)\cdot Z_1(\gamma_3)
   = 21888242871839275222245442326925691337956137906799110242993653357780575606177
\end{aligned}$$

In [ ]:
γ1 = 42
γ2 = 74102
γ3 = 987654321987654321
#Solve here!

#### Second Protocol Version (Succinct, Probabilistic Check)

This improved protocol builds on our earlier version by using:
1. Random challenges sampled by the validator $\gamma_1, \gamma_2, \gamma_3$,
2. Single-point polynomial evaluations to check 
   $ t(\gamma_1) = Q(\gamma_1)\,Z(\gamma_1), \quad
     f_1(\gamma_2) = Q_1(\gamma_2)\,Z_1(\gamma_2), \quad
     f_2(\gamma_3) = Q_2(\gamma_3)\,Z_1(\gamma_3) $.

This lowers the verification cost from a full polynomial multiplication ($O(d^2)$) to an $O(d)$ evaluation, making verification more succinct.


#### Protocol Flow

1. **Compute Fibonacci & Circuit Polynomials**  
   - *(Prover)* Generates $F_4^2$ by building the full circuit table.
   - *(Prover)* Encodes the circuit columns as polynomials $a(x), b(x), c(x)$.

2. **Compute Constraint Polynomials**  
   - *(Prover)* Defines 
      $t(x)= qL(x) \cdot a(x) + qR(x) \cdot b(x) + qM(x) \cdot (a(x) \cdot b(x)) - c(x)$ 
     and finds $Q(x)$ such that 
     $t(x) = Q(x) \cdot Z(x)$.
   - *(Prover)* Similarly, for the recursive constraints:
     $f_1(x) = a(x+1) - b(x),$
     $f_2(x) = b(x+1) - c(x),$
     with quotient polynomials $ Q_1(x), Q_2(x) $ satisfying 
     $f_1(x) = Q_1(x)\cdot Z_1(x)$ 
     and 
     $f_2(x) = Q_2(x)\cdot Z_1(x)$.

3. **Send Polynomials**  
   - *(Prover)* Sends $a(x), b(x), c(x), Q(x), Q_1(x), Q_2(x)$ to the validator.

4. **Verification (Random Challenge & Single-Point Check)**  
   - *(Validator)* Samples random values $\gamma_1, \gamma_2, \gamma_3 \in \mathbb{F}_p$.
   - *(Valdiator)* Computes on its own: 
     $t(\gamma_1), \quad f_1(\gamma_2), \quad f_2(\gamma_3)$
     
   - *(Validator)* Checks:
     $$
     t(\gamma_1) \stackrel{?}{=} Q(\gamma_1)\,Z(\gamma_1), \quad
     f_1(\gamma_2) \stackrel{?}{=} Q_1(\gamma_2)\,Z_1(\gamma_2), \quad
     f_2(\gamma_3) \stackrel{?}{=} Q_2(\gamma_3)\,Z_1(\gamma_3).
     $$
   - By the **Schwartz–Zippel Lemma**, if $t(x) \neq Q(x)\,Z(x)$ (etc.) the probability it also coincides at a random $\gamma$ is at most $\frac{d}{p}$.

5. **Accept or Reject**  
   - *(Validator)* If all checks pass, the proof is accepted; otherwise, it is rejected.

#### Diagram of Protocol

Below is a simplified diagram illustrating these steps:

<center>
  <img src="img_td/succint1_s.png" alt="Succinct Protocol Diagram" width="600">
</center>


With this approach, the **validator** does a constant amount of final checks (polynomial evaluations), regardless of the polynomial degree $d$, achieving **succinct** verification.

## 4.An even more efficient version

To put things into perspective, our latest protocol version requires sharing 6 polynomials of degree at most $d=3$ (quotients are typically shorter), and the validator has to still do at most 3 multiplications per each one of the 3 equality checks. That is a communication cost of about $3*6$ field elements and $3*3$ field element multiplications. In general this would be ca. $d*6$ field elements in communication and $3*d$ field multiplications for the $d$-th Fibonacci number (rounding this to the number of padded rows as discussed before). This does not seem quite useful given that only about $d$ additions would allows us to compute the $d$-th Fibonacci number directly!

But there is a goal behind our strategy. Our goal is to make both the communication and the verification constant ($O(1)$) for circuits with many gates! To achieve this we need a very useful primitive called a Polynomial Commitment Scheme. In particular, we are going to use the method called KZG. This method relies on pairings over elliptic curves and to keep this notebook as lean as possible, we are preparing a separate tutorial on this topic. 

In the following we simply recap some fundamental properties of the needed ingredients while keeping our goal on sight. 

**What do we want to achieve?** In a nutshell what we want to achieve is the following: We want to completely delegate computation of the circuit to the processor, and as before, also the interpolation of the column polynomial and the computation of the quotients. But the twist is, we will not ask for a full copy of a given polynomial $f(x)$, but just a very short and constant size piece of information that we call a **commitment** $c_f=\textsf{commitment}(f(x))$. With this commitment, we can at a later stage challenge the validator to evaluate $f(\gamma)=\alpha$. We will be able to check on our whether indeed $f(\gamma) = \alpha$ by using $\gamma, \alpha$ and the commitment $c_f$ using a function $\textsf{verify}(\alpha,\gamma,c_f)$. 

What this accomplishes:

* Instead of sharing a $d$-size vector for each polynomial over the network, we only share two elements: $c_f \in E$, where $E$ is an elliptic curve, and $\alpha \in \mathbb{F}_p$.
* A malicious or misbehaving processor $P$ cannot adapt to the interactive challenge $\gamma$ since it has already commited to a polynomial before seeing the challenge. This gives the validator confidence that the value was correctly computed.
* We can directly perform the constraint checks $f(\gamma)=Q(\gamma) \cdot Z(\gamma)$ in constant time, regardless of the number of gates in the circuit (that is, the polynomial $d$ of the interpolating polynomial). Certainly there are some upper bounds to the size of a circuit and we will come back to this later.

Given that by making this protocol more efficient we have to make it more resilient to misbehaving processors, we are going to call $P$ a *prover* and $V$ a *verifier* from now on.

#### Pairings 
The first ingredient for our new protocol version will be a suitable elliptic curve pairing. We will give you some precomputed functions, but you may reconstruct them on your own.

We need a mapping:

$$e: \mathbb{G_1} \times \mathbb{G_2} \to \mathbb{G_t}$$

for additive cyclic groups $(\mathbb{G_1},+)$ and $(\mathbb{G_2},+)$ and multiplicative $(\mathbb{G_t},*)$. Note that typically the first two groups are additive (since they are subgroups of an elliptic curve) and the destination group is multiplicative (usually defined over a field).

The mapping $e$ we are looking for is "bilinear", that means:

$$e(P + P', Q) = e(P,Q)*e(P',Q)$$
$$e(P,Q+Q')=e(P,Q)*e(P,Q')$$

To construct **secure** pairings, it is important to chose the groups in a particular way which motivates choosing isomorphic but disjoint $\mathbb{G_1},\mathbb{G_2}$ which can be found as subgroups of an elliptic curve over a finite field extension $E(\mathbb{F}_{q^k})$ and $\mathbb{G_t}$ a subgroup of $\mathbb{F^*}_{q^k}$. We are not going to explain this in more depth but if you are curious check [Pairings for Beginners by Craig Costello](https://static1.squarespace.com/static/5fdbb09f31d71c1227082339/t/5ff394720493bd28278889c6/1609798774687/PairingsForBeginners.pdf).



In [7]:
import kzg
def e(P,Q):
    #Base field for BN254
    q = 21888242871839275222246405745257275088696311157297823662689037894645226208583
    #Number of points on BN254
    n = Integer(21888242871839275222246405745257275088548364400416034343698204186575808495617)
    k = 12
    t = 6*pow(4965661367192848881,2)+1
    return P.ate_pairing(Q, n, k, t,q)

In [8]:
P = kzg.P
Q = kzg.Q

show("P:",P)
show("Q:",Q)
show(e(P,Q))

'P:' (1 : 2 : 1)

'Q:' ((11559732032986387107991004021392285783925812861821192530917403151452391805634*ubar + 10857046999023057135944570762232829481370756359578518086990519993285655852781)*wbar^2 : (4082367875863433681332203403145435568316851327593401208105741076214120093531*ubar + 8495653923123431417604973247489272438418190587263600148770280649306958101930)*wbar^3 : 1)

(15485169958187804055419367470727586316504614054038886924834910714644298081076*ubar + 14110042394329134992063060126132667374513734777529849604059403292605387766763)*wbar^5 + (17576687079377474679994084448626848766364851367692113922904391421785791556046*ubar + 15263055409656443157285976462204135910932331054849227253916602565546649989931)*wbar^4 + (1451055192040231810539564576166577843622781952620338854414099592814058446945*ubar + 19362144781956138062795090409372908218130916662778620452325203445403259212772)*wbar^3 + (7786695150901409298856750468044088494675441502192568573069643047819302766865*ubar + 16893543948180465298099197784510869270928575391710795413610320310782594693022)*wbar^2 + (8339301614695659949019663159218383351646785965635781625471751346963751888108*ubar + 465898347631256865746797445390333094396673986355153352843254722736295902148)*wbar + 15780004297944302906621723394852439600559246581555156000423333429824441348714*ubar + 17296046054077859324401147072502722261136321305912126349272372680482680769943

**Exercise 4** Check that the provided $e$ function is bilinear by randomly sampling an integer $s$ and multiplying over the additive subgroups:

$$ e([s]\cdot P,Q)=e(P,[s] \cdot Q)=e(P,Q)^s$$



In [70]:
#Solve here!
s=


#### KZG commitment 

Our goal is to leverage pairings to help provers to *commit* to a polynomial $f(x)$ before receiving the challenge $\gamma$ and for the verifier to have (cryptographic) certainty that the evaluation $f(\gamma)$ was done on this same commited polynomial, ruling out adaptive attacks. There are several answers in the literature to solve this exact problem, one prominent one is the [KZG commitment scheme](https://www.iacr.org/archive/asiacrypt2010/6477178/6477178.pdf). We can illustrate the protocol with the following sequence diagram.

<center><img src="img_td/kzg.png" width="300"><figcaption><b>Figure 3</b>: The KZG commitment scheme. </figcaption></center>
<br>

**Exercise 5**
For this protocol to work, we need a trusted setup based on a uniformly random (and secret) *toxic waste* τ. This is called toxic waste since it must be deleted after some setup values have been computed with it in order for the protocol to be secure. We also need to declare a public maximum polynomial degree $l$. Since we will later commit to multiplication of low degree polynomials, let us pick $l=10$.

With example $\tau=424242$, compute two public parameters: a vector $$S_1 = [  P, \tau \cdot P , \tau^2 \cdot P , \cdots,  \tau^{l} \cdot P]$$

where $P$ is the generator of $\mathbb{G}_1$ and a value $$S_2 = \tau \cdot Q$$ where $Q$ is the generator of the group $\mathbb{G}_2$.

In [ ]:
#Solve here!
τ = 424242
l=10
S1=
S2=

If computed correctly you should get:

$S1 =\left[\left(1 : 2 : 1\right), \left(3388106087484702502772161951837520865682365219266094058038991331506090320735 : 10310862820936104897688292475187545406556140958479334534497508049176340462168 : 1\right), \left(5786299459695789788261684662131390109566181344091061986154006773045920884529 : 18198083969462810098554436542449595852850793038741994323649111219383738839374 : 1\right), \left(5932386722799341949736906223819928737327798404444018663115122905353930163217 : 17316789122660091087982740003020863959605853926224865905583913597777031548221 : 1\right), \left(14603143723753528702641274659350642057392810682423113693594196367092666272615 : 17734357240639322091230265203514580412414342326449923671483947891721675198742 : 1\right), \left(1485870938289488242176031429247482114928668541241910486620153058173498550422 : 8975504733770431670587733015492501696212039808343155103983507429794936273090 : 1\right), \left(35087711382470900519387874116184362249603265567233267930080986182721755391 : 9687295787051875541592859447450503637855174457649076441192556983747473888473 : 1\right), \left(21100087779353388367643433264506322944085867030119306347233967681812330825611 : 17068231115940713163151138839050846284784298672314356795840059410045954257490 : 1\right), \left(10641104735982477671491147382199353108251699282959504595440637553392719424596 : 3033897669229592557681916129851961240665115699542840722127903032620063452401 : 1\right), \left(12461517031230393262909098092888598630825065680164551109453025306556394017248 : 11406184524960241674372900569663181593279300027508257400554857135388045542548 : 1\right), \left(2310703006113265329769149621286389443406062880925858119855767049953176497692 : 20254007398902051444166009223523085479917136359422165955249757385592434525697 : 1\right)\right]
$

and:

$S2 = \left(\left(18178798121214140434976027337622332703345638858169149597321922257824015358086 ubar + 17892288729126503148648989889095246645011224085306750203724686651395006961383\right) wbar^{2} : \left(15251007341879318938631029793693405432107362852215428951388705343135014132495 ubar + 11565079700697045165214936027269859661189920603201791930010463312628382919842\right) wbar^{3} : 1\right)$

<center><img src="img_td/sendcommitment.png" width="300"></center>
<br>

**Exercise 6** Implement a *commitment* function. This function takes as parameters the public vector $S_1$ we computed before and the polynomial $p(x)$ we want to commit to. It then computes $$\sum_{i=0}^{l} a_i \cdot S_1[i] $$

Note that by construction this is:

 $$a_0 \cdot P + a_1 \cdot \tau \cdot P + a_2 \cdot \tau^2 \cdot P  + \cdots + a_{l} \cdot \tau^{l} \cdot P $$

 which is equivalent to:

 $$p(\tau) \cdot P$$

 Note that this will be a point on the curve $EK$. Test your function over the polynomial $a(x)$ of the Fibonacci example.

In [ ]:
def commitment(S1,p):
    #Solve here!
    return c

c = 

If computed correctly, you should get a commitment:

$$c = \left(19772988533509128204934208855583299243034568734268587639600165428945857082832 : 21198549198844316278609987090510616007059968516882705878791833422348082226388 : 1\right)$$


Now this commitment $c$ can be given to the verifier before any challenge is issued. 


<center><img src="img_td/challengephase.png" width="300"></center>


In a subsequent step, the prover will receive the challenge $\gamma$ from the verifier. They will now give back $f(\gamma)$ but this time also a proof $\pi$ that indeed this is the evaluation of $\gamma$ in the very same polynomial that the prover commited to through $c$.

**Exercise 7** The $\textsf{proof}$ generating function receives two parameters, the vector $S_1$ and a polynomial $Q_c$. Similar as before, this will compute a point on the curve which is equal to:

 $$\pi = b_0 \cdot P + b_1 \cdot \tau \cdot P + b_2 \cdot \tau^2 \cdot P + \cdots + b_{l} \cdot \tau^{l} \cdot P = Q_c(\tau) \cdot P$$

where $b_i$ are the coefficients of the polynomial $Q_c$. Now, the way this polynomial is constructed is essential for the proof:

$$Q_c(x) = \frac{f(x) - f(\gamma)}{x-\gamma} $$


Again here we are exploiting the fact that $\gamma$ is a root of $f(x) - f(\gamma)$ therefore this polynomial must be divisible by $x-\gamma$. Compute the $\texttt{proof}$ function and test it on the $Q_c$ polynomial for the challenge $\gamma = 151515$ and the polynomial $a(x)$.

In [ ]:
def proof(S1,Qc):
    #Solve here!
    return c

#Given a challenge γ
γ = 151515
#Solve here!
b = 
Qc= 
π = 

If computed correctly, the proof obtained from $\texttt{proof}$ when called on $Q_c$ should be:

$$\pi = \left(18427764633036746853571353448280965399000717707037995558464363091930831203059 : 15823869948268955546174436172082876809321127559750802916163692992123652869945 : 1\right)$$

The value of the polynomial on the challenge point is:

$$a(\gamma) = 1739069066686765$$

<center><img src="img_td/verificationphase.png" width="300"></center>


So now the prover, who has issued the challenge $\gamma$, has both $c$ and $\pi$. He will use the verification function that takes as parameters $c, \pi, \gamma$ and $b=f(\gamma)$. We are finally ready to use the pairing. We want to check whether the following values are equal.

$$
{e(\pi, S_2 - \gamma \cdot Q)} \stackrel{?}{=} {e(c- b \cdot P, Q)}
$$

The **left-hand side** is:

$$
{e(\pi, S_2 - \gamma \cdot Q)} = e( Q_c(\tau) \cdot P, (\tau - \gamma) \cdot Q)
$$

and using bilinearity:

$$
e( Q_c(\tau) \cdot P, (\tau - \gamma) \cdot Q) = e( {Q_c(\tau)\cdot (\tau - \gamma)} \cdot P,  Q)
$$

The **right-hand side** is:

$$
{e(c- b \cdot P, Q)} = e(f(\tau) \cdot P - f(\gamma) \cdot P, Q) = e( {(f(\tau) - f(\gamma))} \cdot P, Q)
$$

So essentially what the pairing allows us to do is to check:

$$
{f(\tau) - f(\gamma)} \stackrel{?}{=} {Q_c(\tau)\cdot (\tau - \gamma)}
$$

which is expected by the construction of $Q_c$. This is equivalent to checking polynomial equality using the challenge $\tau$. But remember, nobody knows $\tau$ directly because we deleted it after setup ("toxic waste")!

**Exercise 8** Implement a $\textsf{verification}$ function that takes $c,\pi,\gamma$ and $f(\gamma)=b$ as parameters. It should output true or false, by evaluating:

$${e(\pi, S_2 - \gamma \cdot Q)} \stackrel{?}{=} {e(c- b \cdot P, Q)}
$$

Test the function on the previously computed commitment $c$ for $a(x)$ and the proof $\pi$ computed in the previous exercise.

In [ ]:
def verification(c,π,γ,b):
    #Solve here!
    
print(verification(c,π,γ,b)) #should return True

#### New Protocol Version

We divide the protocol into **Setup**, **Prover Computation**, **Verifier Challenge**, 
and **Proof & Verification** phases.  



#### 1. Setup Phase

1. *(Both)* Agree on the public parameters $(S_1, S_2)$.
2. *(Verifier)* Publicly specifies the maximum polynomial degree $d$, if needed.
3. *(Trusted Ceremony / Setup)* Generates a secret $\tau$ and produces $(S_1, S_2)$.  
   After this step, $\tau$ (the toxic waste) must be discarded to maintain security.



#### 2. Prover Computation Phase

1. *(Prover)* Interpolates the column polynomials 
   $a(x), b(x), c(x)$.
2. *(Prover)* Constructs quotient polynomials 
   $Q(x), Q_1(x), Q_2(x)$
   with respect to the vanishing polynomials $Z(x), Z_1(x)$.
3. *(Prover)* Commits to each polynomial by computing
   $$
   c_a = \text{commit}(a), \quad
   c_b = \text{commit}(b), \quad
   c_c = \text{commit}(c), \quad
   c_Q = \text{commit}(Q), \quad \dots
   $$
   and sends these commitments to the Verifier.



#### 3. Verifier Challenge Phase

1. *(Verifier)* Samples random challenges 
   $\gamma_1, \gamma_2, \gamma_3$.
2. *(Verifier)* Sends the challenges to the Prover.



#### 4. Proof & Verification Phase

1. *(Prover)* Evaluates each committed polynomial at its corresponding challenge. For example,
   $$
   a(\gamma_1), \quad b(\gamma_1), \quad c(\gamma_1), \quad Q(\gamma_1).
   $$
   Then constructs proofs $\pi_a, \pi_b, \pi_c, \pi_Q$ via the KZG evaluation method.
   The Prover sends 
   $$
   \bigl(a(\gamma_1),\,b(\gamma_1),\,c(\gamma_1),\,Q(\gamma_1)\bigr)
   $$
   and
   $$
   (\pi_a,\,\pi_b,\,\pi_c,\,\pi_Q)
   $$
   to the Verifier.

2. *(Verifier)* Uses the commitments 
   $(c_a,\,c_b,\,c_c,\,c_Q)$
   and the proofs $\pi_a, \pi_b, \pi_c, \pi_Q$ to verify each polynomial evaluation.  
   It then checks the constraint
   $$
   a(\gamma_1) + b(\gamma_1) - c(\gamma_1) 
     \;=\; Q(\gamma_1)\,\cdot Z(\gamma_1).
   $$
   Since the Verifier can construct $Z(x)$ itself, it only needs to check
   $t(\gamma_1) = Q(\gamma_1)\,Z(\gamma_1)$, 
   where $t(x)= qL(x) \cdot a(x) + qR(x) \cdot b(x) + qM(x) \cdot (a(x) \cdot b(x)) - c(x)$.

3. *(Verifier)* Repeats a similar check for the recursive constraints:
   $$
   a(\gamma_2 + 1) \;-\; b(\gamma_2) \;=\; Q_1(\gamma_2)\,\cdot Z_1(\gamma_2),
   $$
   $$
   b(\gamma_3 + 1) \;-\; c(\gamma_3) \;=\; Q_2(\gamma_3)\,\cdot Z_1(\gamma_3).
   $$

4. *(Verifier)* If all these checks pass, the Verifier accepts. Otherwise, it rejects.



As you can see, both the number of message exchanges and the operations performed by the Verifier are now almost constant. In other words, 
the Verifier can ask the Prover to compute $F_6$ or $F_{1000}$ and still carry out about the same number of computations. The only operation dependent on the number of rows of the circuit on the verifier's side is the computation of the vanishing polynomials $Z$ and $Z_1$.

#### Protocol Diagram

The following diagram visually represents the four main phases of the protocol:

1. **Phase 1: Setup**  
   - Both parties agree on the public parameters $(S_1, S_2)$.  
   - The Verifier may specify the maximum polynomial degree $d$.  
   - A trusted ceremony generates the secret $\tau$, then discards it, while producing the public parameters $(S_1, S_2)$.

2. **Phase 2: Prover Computation**  
   - The Prover interpolates the circuit columns (e.g., Fibonacci) into polynomials $a(x), b(x), c(x)$.  
   - Constructs the quotient polynomials $Q(x), Q_1(x), Q_2(x)$ relative to vanishing polynomials $Z, Z_1$.  
   - Commits to these polynomials (e.g., $c_a, c_b, c_c, c_Q, \dots$) and sends the commitments to the Verifier.

3. **Phase 3: Verifier Challenge**  
   - The Verifier randomly samples challenges $\gamma_1, \gamma_2, \gamma_3$.  
   - Sends these challenges to the Prover.

4. **Phase 4: Proof & Verification**  
   - The Prover evaluates each committed polynomial at the corresponding $\gamma$ value and constructs proofs (e.g., $\pi_a, \pi_b$).  
   - Sends the polynomial evaluations and proofs to the Verifier.  
   - The Verifier checks that the evaluations match the commitments, confirming equations like 
     $$
     t(\gamma_1) \;=\; Q(\gamma_1) \cdot Z(\gamma_1).
     $$
   - If all checks pass, the Verifier accepts; otherwise, it rejects.



<center><img src="img_td/succint2.png" width="500"></center>

## 5.Domains and selectors

Note that so far we have exploited a convenient structure to verify the correctness of a computation: that of evaluating polynomials on a set of points (or indexes) $I$ and checking constraints by means of quotient polynomials. This way we could almost bring the computational time of the verifier to a constant. But there is a catch: computing the vanishing polynomials $Z$ and $Z_1$ as the multiplication of $(x-i)$ monomials, although not a very expensive operation is still dependent on the number of rows of the circuit (which corresponds to the maximum polynomial degree $d$). Also, the way we check constraints $f_1$ and $f_2$, by comparing values in a given row with the next row, and by considering a different index set $I' = I \setminus{\{3,4\}}$ is not very general. What if we want to evaluate a circuit where connections between gates are different, do we have to define a new polynomial for each constraint by hand and carefully think about which domain to evaluate? 

To address this issues, and to improve efficiency even more, let us introduce a different type of indexing: using a multiplicative subgroup (also referred to commonly as a *domain*). A particularly interesting set of domains are the ones with size $n=2^k$. Mathematically we want to find a set of points $\Omega \in \mathbb{F}_p$ such that:

$$\Omega = \{ \omega, \omega^2, \dots, \omega^{2^k-1},1 \}$$

for some generator $\omega \in \mathbb{F}_p$. This set will replace our previous set of indexes $I$. One immediate advantage of this choice is that computing the vanishing polynomial for the domain can be done in constant time, because $Z = x^n - 1$, and we no longer need to multiply each monomial $(x-i)$ as we did with the additive indexes. This follows immediately from the definition of $\Omega$ given that each element is a so-called "$n$-root of unity", a field element of order $n$: for every $\omega^i \in \Omega$, it holds $(\omega^i)^n = (\omega^n)^i = 1^i = 1$.

**Exercise 9** Find a generator $\omega$ of a domain of order $4$ using the following algorithm. Consider $r = (p-1)/4$. Find the smallest $h \in \mathbb{F}_p$ such that $\omega=h^r$ has multiplicative order $4$. In other words, $\omega^4 = 1$ and $\omega^i \neq 1$ for $i \in \{1,2,3,4\}$.

In [ ]:
n = pow(2,2)

#Solve here!
r = 
h = 
ω = 



If computed correctly you should obtain:

$$\omega = 21888242871839275217838484774961031246007050428528088939761107053157389710902
$$

Note that a further interesting property of working with multiplicative indexes is that they "wrap around", that is, whenever we go beyond the order of the multiplicative group, we cycle back to the beginning of the group: $\omega^0 = \omega^n = 1$. We will benefit from this property in the next section, were we generalize wiring constraints.


**Exercise 10** Interpolate the value vectors and selector vectors as before, now over $\Omega$ instead of $I$.

In [ ]:
#Solve here! Hint: your previously defined interpolate function should also work here!
Ω = 
a = 
b = 
c = 

qL =
qR =
qM =


If computed correctly you should have obtained:

$$a(x)=16416182153879456417786784551517017277046601793284012108757927423286461067892 x^{3} + 5472060717959818805561601436314318772137091100104008585924551046643952123905 x^{2} + 5472060717959818804459621193740257811501762607132022234940276763289347427726 x + 16416182153879456416684804308942956316411273300312025757773653139931856371714
$$

We said that one of the advantage of working with multiplicatives domains is that the verifier can now use $Z(x)=x^n-1$ as the vanishing polynomial, which can be computed in constant time, instead of $Z(x)=\prod_i^d (x-i)$ which is an $O(d)$ operation. 

**Exercise 11**
Let  $t(x) \;=\; qM*a*b + qL*a+qR*b-c$ where the polynomials $a(x),\,b(x),\,c(x)$ are obtained by Lagrange–interpolating the three witness columns over the multiplicative domain  $\Omega \;=\; \{\omega,\; \omega^{2}, \dots, \omega^{4}\,\}$. Take the domain‑vanishing polynomial  $Z(x) \;=\; x^{4} - 1$. Perform polynomial long‑division of $t(x)$ by $Z(x)$; that is, find polynomials $Q(x),\,R(x) \in \mathbb{F}_{p}[x]$ such that:

$$t(x) = Q(x) \cdot Z(x) + R(x)$$

Verify that the division is *exact*: show that the remainder $R(x)$ is zero. Equivalently, confirm that there exists a polynomial $Q(x) \in \mathbb{F}_{p}[x]$ for which  

$$t(x) \;=\; Q(x) \cdot Z(x)$$

In [ ]:
#Solve here!

t = 
Z =                      
Quo=
Rem=

print("Quotient Q(x):\n", Quo)
print("\nRemainder R(x):\n", Rem)
print("\nIs remainder zero?  ", Rem == 0) #should be True

## 6.Copy constraints

Recall our discussion on the recursive constraints $f_1,f_2,f_3$ and $f_4$ for our squared Fibonacci example. What we wanted to achieve essentially was to ensure that some values (input or output) in a previous row were re-used in a subsequent row. This is nothing else than what is commonly known as the "wiring" of a circuit: an input wire may be linked to multiple gates, or the output wire of a gate can be the input of another. Mathematically we can think of wiring as a partition of positions in the table representing the circuit into equivalence classes (defined by equality). Consider the following image. Here we can see the 4 partitions as different colors both on the table and on the circuit. 

<center><img src="img_td/partitions_s.png" width="400"></center>




**Exercise 12**: Compute an array encoding a permutation over the three columns $a,b,c$. That is, compute a mapping from $1,2,\cdots, 12$ to $1,2,\cdots, 12$ that encodes the equivalence relation of the wiring between the three columns. For example $2 \to 5$ and $5 \to 2$ because 5 is the first position ($(4+1)$) of the second column. 

In [ ]:
sigma = {}
#First input
sigma[1] = 1

#First cycle
sigma[2] = 5
sigma[5] = 2

#Solve here!

If implemented correctly, your permutation should pass the following checks.

In [18]:
print("Length should be 12:", len(sigma) == 12)
print("There should be no repeated elements in the domain:", len(set(sigma.keys())) == 12 ) 
print("Both the image and domain should be the indexes from 1 to 12", set(sigma.keys()) == set(range(1,13)) == set(sigma.values()))
print("The cycle (3,4+2,8+1) should be in sigma:" , sigma[3] == 4+2 and sigma[4+2] == 8+1 and sigma[8+1] == 3)

Length should be 12: True
There should be no repeated elements in the domain: True
Both the image and domain should be the indexes from 1 to 12 True
The cycle (3,4+2,8+1) should be in sigma: True


The mathematical property we want to check for a given function $f: I \to \mathbb{F}$ is the following:

$$\forall i \in I: f(i)= f(\sigma(i))$$

If this holds, we know that all values in our equivalence classes are equal. We can in fact check this for our permutation $\sigma$ and our three columns $a,b,c$ as follows.



In [19]:
columns = [a, b, c]
for col in range(3):
    for i in range(1, n+1):                      # rows 1…n
        slot = col*n + i                         # “flattened” slot in 1…3n
        mapped = sigma[slot]                     # in 1…3n

        # convert to 0-based so divmod works nicely
        zero_slot = mapped - 1                   
        col2, idx2_zero = divmod(zero_slot, n)   # col2 in 0..2, idx2_zero in 0..n-1
        i2 = idx2_zero + 1                       # back to 1…n indexing
        
        x1 = ω**(i )
        x2 = ω**(i2 )

        assert columns[col](x1) == columns[col2](x2), "Permutation property does not hold"
            

Now the question is, how does the prover convince the verifier that this holds without sharing all intermediate computation values?

One simple way to achieve this is to run the equality testing protocol for:

$$ f \circ \sigma - f = 0 $$

over the domain $\Omega$ and an interpolated version of $\sigma$. Unfortunately, this solution is inefficient since the composition of two polynomials of degree $n$ has degree of about $n^2$. For instance if we compose $a$ and $b$ we obtain a polynomial of degree 9 (why?):

In [20]:
a(b)

21802741923121153053822748313780136717971970536841404091162235932729973504748*x^9 + 17698696384651288949806513208156374961458495281087583232325986218843750697220*x^8 + 15219168871825746054207931767645017665721029176757758524459966167252422127810*x^7 + 13851153692335791356915520435638727170049384234971330620701598396816370310749*x^6 + 15732174564134479069295544857125849351800122391714983737485907109165176444762*x^5 + 6156068307704796159975984934541064360798461151397463593737045633796236988998*x^4 + 21717240974403030896832135899008880813987109787851132230088113368688162236738*x^3 + 12141134717973347974460997478437561320460776708896648741869253349838123410604*x^2 + 13081645153872691828051025054357947332337167624162695003407885568722756022565*x + 15817675512852601238803964089886828980501917740558913054272020360688200433438

Instead, we are going to introduce a new protocol to approach this check in an efficient way. Fundamentally, what we want to do is to capture the "for all" quantifier where we check $f(\sigma(i))$ with $f(i)$ using a product, in a way that we avoid computing the costly polynomial composition.

To motivate the concrete algorithm, consider the following. Assume $h$ is a cryptographically secure hash function. If we compute

$$GP = \prod_{i=1}^n \frac{h(i,f(i))}{h(i,f(\sigma(i)))}$$

we expect to be $GP=1$ if and only if $f(\sigma(i))=f(i)$. One direction is easy to see, for the other one we would conclude that either there was a hash collision or some unlikely scenario arises (details are left as an exercise).

One useful observation is that we can write the $GP$ alternatively like this:

$$GP = \prod_{i=1}^n \frac{h(i,f(i))}{h(i,f(\sigma(i)))} = \prod_{i=1}^n \frac{h(i,f(i))}{h(\sigma(i),f(i))} $$

because the values are in the equivalence class defined by $\sigma$. In fact, the numerator is exactly the same, and the denominator as well: noticed that on the original denominator we had $h(\sigma(i), f(\sigma(i))$ which, if the permutation holds, is equal to $h(\sigma(i),f(i)$. But the advantage of this equivalent representation is that we do not have to explicitely compose the polynomial $f$ with $\sigma$.

Now, in practice we want to have an efficient "hash" function that has the same security properties as before, such that $GP=1$ implies (with high probability) that the permutation property holds. For this we consider the class of functions:

$$h(i,f(i)) = i + \beta \cdot f(i) + \gamma$$

for random interactive challenges $\beta$ and $\gamma$. It is possible to show using Schwartz-Zippel that those will behave similar to the cryptographic hash function in our discussion above, but they are efficiently computable (for more details see the original paper).

We will now step by step construct the argument that takes advantage of this. First we start by defining a numerator and denominator functions, and their respective accumulators.

**Exercise 13** Compute the numerator using the formula $$ (column - 1) \cdot n + i + \beta\cdot f(\omega^i) + \gamma$$ and the denominator as $$\sigma(column - 1) \cdot n + i + \beta\cdot f(\omega^i) + \gamma$$

In [ ]:
def numerator(i, column,f,sigma,beta, gamma):
#Solve here!    
    return value    

def denominator(i, column,f,sigma,beta, gamma):
#Solve here!
    return value       



In [22]:
#Test cases
numerator(3,1,a,sigma,42,42) == 87
denominator(3,1,a,sigma,42,42) == 90

numerator(6,2,b,sigma,42,42) == 94
denominator(6,2,b,sigma,42,42) == 91

True

**Exercise 14** Compute accumulator functions for the numerator and the accumulator respectively as:

$$ \textsf{acc\_numerator}(i, column,f,\sigma,\beta, \gamma) = \prod_{j=1}^{i-1} \textsf{numerator}(j, column,f,\sigma,\beta, \gamma)$$
$$ \textsf{acc\_denominator}(i, column,f,\sigma,\beta, \gamma) = \prod_{j=1}^{i-1} \textsf{denominator}(j, column,f,\sigma,\beta, \gamma)$$

In [74]:
def acc_numerator(i, column,f,sigma,beta, gamma):
    value = 1 
    #Solve here!
    return value    

def acc_denominator(i, column,f,sigma,beta, gamma):
    value = 1 
    #Solve here!
    return value        

If implemented correctly, the following check should pass.

In [24]:
N_n = acc_numerator(n+1,1,a,sigma,42,42)*acc_numerator(n+1,2,b,sigma,42,42)*acc_numerator(n+1,3,c,sigma,42,42)
D_n = acc_denominator(n+1,1,a,sigma,42,42)*acc_denominator(n+1,2,b,sigma,42,42)*acc_denominator(n+1,3,c,sigma,42,42)
print("When multiplying z for each column with the corresponding upper and lower bounds one should obtain 1:", N_n//D_n==1)  

When multiplying z for each column with the corresponding upper and lower bounds one should obtain 1: True


Now that we have both the numerator and denominator functions, and the partial accumulators we can proceed to spell out the interactive protocol that allows the (potentially malicious) prover to prove that the permutation holds over the three columns $a$,$b$,$c$ in an efficient way. Note that the following function has an interesting property, which is that it can be defined recursively:

$$z(i+1)= \frac{\mathsf{acc\_numerator}(i+1)}{\mathsf{acc\_denominator}(i+1)} = z(i) \cdot \frac{\mathsf{numerator}(i)}{\mathsf{denominator}(i)}$$

Moreover, the base case is $z(1)=1$. We would like also to be able to prove that $z(n+1)=1$, given that this represents the grand product argument when considering all indexes. 

We can develop a succinct argument using polynomials by following these steps:

* We assume the prover has already commited to $a,b,c$.
* The verifier challenges the prover with random $\beta$ and $\gamma \in \mathbb{F}_p$.
* The prover interpolates $z$ over $\Omega$ by computing each of the points for $i=1, \dots, n$.
* The prover interpolates polynomials $N$ and $D$ corresponding to the $\mathsf{numerator}$ and $\mathsf{denominator}$ functions.
* The prover computes the quotient polynomial $Q_z$ for the polynomial $z\cdot N -D\cdot z(x \cdot ω)$ and commits to it.
* The verifier checks the following two properties
    * $z(\omega)=1$
    *  $z\cdot N -D\cdot z(x \cdot ω)=0$

It is interesting to note that in the multiplicative domain, $z(\omega)=1$ automatically implies $z(\omega^{n+1})=1$. This follows from:

$$z(\omega^{n+1})=z(\omega^n \cdot \omega) = z(\omega) =1 $$

**Exercise 15** Compute a function that receives as parameters the three polynomials $a,b,c$ , the challenges $\beta, \gamma$ and the domain $\Omega$ and returns the interpolation fo the $z$, $N$ and $D$ polynomials.

In [ ]:
def interpolate_z_N_D(a,b,c, beta, gamma, Ω):
#TBD
    return z,N,D

z,N,D = interpolate_z_N_D(a,b,c,42,42,Ω)

If computed correctly, the interpolated functions should pass the $z(\omega)=1$ check as follows.

In [26]:
L1 = prod((x - ω**m)//(ω - ω**m) for m in range(2,n+1))
L1=R(L1)
ZH = R(x^n-1)
(ZH).divides(L1*(z-1))

True

Moreover it should pass the following check, which proves the recursive constraint.

In [27]:
(ZH).divides( z*N -D*z(x*ω))

True

The polynomial $z(x) \cdot N(x) -D(x) \cdot z(x*ω)$ is:

$$z(x) \cdot N(x) -D(x) \cdot z(x*ω) = 19088033315219631021107812019832839306780395371633216223270161982137379434783 x^{6} + 9442345120481651903520919226899454023736002770233426083846080860317389062734 x^{5} + 21082329243894486516657209903907612685168280090308035096051805040844722189074 x^{4} + 2800209556619644201138593725424435781767969028782818120428042204438429060834 x^{2} + 12445897751357623318725486518357821064812361630182608259852123326258419432883 x + 805913627944788705589195841349662403380084310107999247646399145731086306543
$$

In [28]:
show(z*N -D*z(x*ω))

19088033315219631021107812019832839306780395371633216223270161982137379434783*x^6 + 9442345120481651903520919226899454023736002770233426083846080860317389062734*x^5 + 21082329243894486516657209903907612685168280090308035096051805040844722189074*x^4 + 2800209556619644201138593725424435781767969028782818120428042204438429060834*x^2 + 12445897751357623318725486518357821064812361630182608259852123326258419432883*x + 805913627944788705589195841349662403380084310107999247646399145731086306543

## 7.Towards ZKP

Note that although we have focused on a malicious prover so far, the verifier may deviate from the honest protocol to learn secret data from the verifier. For example, they can choose adaptive off-domain points or collude across multiple openings with the goal of extracting intermediate witness values or private inputs. 

### Blinding via vanishing-polynomial multiples

To defend against this, we mask each witness polynomial \(f(x)\)  by adding a random multiple of the vanishing polynomial.
$$Z_H(x) \;=\; x^n - 1.$$

Concretely, before committing we form

$$\widetilde f(x) \;=\; f(x)\;+\;Z_H(x)\cdot p(x),$$

where $p(x)$ is drawn uniformly at random from all polynomials of degree $<k$ (for some security parameter $k$.

### One fresh blinding factor per opening

Each time the verifier opens $\widetilde f$ at a point $z\not\in\{\omega^i\}$, they learn

$$\widetilde f(z) \;=\; f(z)\;+\;Z_H(z)\cdot p(z)$$

which yields one linear equation in the coefficients of $p(x)$.  
- If the same $p(x)$ were reused for multiple openings, a malicious verifier could gather enough of these equations to solve for $p(x)$, then subtract $Z_H(x)\cdot p(x)$ and recover the original $f(x)$.  
- To preserve zero-knowledge, we therefore allocate one independent random polynomial $p(x)$ for each distinct off-domain opening query.

*Example* Consider the following example for blinding the witness polynomial $a(x)$. We assume two openings on $a(x)$, as we will see later in the full protocol.

In [29]:
k = 2

p = R.random_element(degree=k-1)
print("p(x) =", p)
a_blind = a + p*ZH
show(a_blind)
print("Degree of a_blind:",a_blind.degree())

p(x) = 7846974345743743743009628096037841263957128127461006184358032298078777836753*x + 17400493923130958490506171015032528352276895896363983356660864143955303261126


7846974345743743743009628096037841263957128127461006184358032298078777836753*x^5 + 17400493923130958490506171015032528352276895896363983356660864143955303261126*x^4 + 16416182153879456417786784551517017277046601793284012108757927423286461067892*x^3 + 5472060717959818805561601436314318772137091100104008585924551046643952123905*x^2 + 19513329244055350283696398842959691636092998880087050394280448651786378086590*x + 20903931102587773148425039039167703052682741804364076744810993182552361606205

Degree of a_blind: 5


**Exercise 16** Assuming $b(x),c(x)$ also have at most two openings, blind them similarly to $a(x)$. Afterwards, interpolate $z\_poly, N\_poly, D\_poly$ using the given $\beta$ and $\gamma$. Finally blind $z\_poly$ assuming also at most 2 openings. 

In [ ]:
beta = gamma = 42

b_blind = #?
c_blind = #?
z_poly = #?
N_poly = #?
D_poly = #?
z_poly_blind = #?

If computed correctly the following check should still pass on the blinded polynomials, because the roots of the polynomial do not change (points of the domain).

In [32]:
assert (ZH).divides( z_poly_blind*N_poly -D_poly*z_poly_blind(x*ω)) == True, "Something went wrong"

## 8.Fiat Shamir 
Now let's put all the pieces together.

In practice, we often want a non-interactive proof system so that we can compute the proofs at one point in time on a given machine, and check them later when needed. To achieve this, we use the Fiat-Shamir transformation, where we assume a hash function behaves as a random oracle. The idea is that the prover himself can generate "unbiased" challenges based on the messages he would send to the verifier in an interactive version of the protocol by using a hash function. See for instance the following two functions that operate over a string that we will call a "transcript". 

The first function $\textsf{push}$ adds values to the transcript (field elements, curve elements) by using delimiters to separate them. The $\textsf{generate\_challenge}$ function will hash a given transcript and create an element in $\mathbb{F}$ from the hash.

In [31]:
import hashlib

def push(v, transcript, delim="|"):
    return transcript + delim + str(v)


def generate_challenge(transcript):
    # Encode the transcript string into bytes for the hash function
    transcript_bytes = transcript.encode('utf-8')
    
    # Compute the SHA256 hash and get its hexadecimal representation
    sha256_hash = hashlib.sha256(transcript_bytes).hexdigest()
    
    # Convert the hexadecimal hash string to an integer
    hash_int = int(sha256_hash, 16)
    
    # Cast the integer into an element of our finite field F
    return F(hash_int)

#Examples

transcript = ""
print(f"Initial transcript: '{transcript}'")

# Push a number and an elliptic curve point to the transcript
transcript = push(12345, transcript)
transcript = push(kzg.P, transcript)
print(f"Updated transcript: '{transcript}'")

# # Generate a challenge from the transcript
challenge = generate_challenge(transcript)
print(f"Generated challenge: {challenge}")

Initial transcript: ''
Updated transcript: '|12345|(1 : 2 : 1)'
Generated challenge: 19685844868109878423961294678126802425998569544130367002668967398805728209250


**Step 1** We start with an empty transcript and add the initial values $a(\omega)$ and $b(\omega)$ which should be $0$ and $1$ to the transcript, as well as the output $c(\omega^4)$ which should be 9. We also add commitments to the blinded polynomials $a\_blind, b\_blind, c\_blind$.

**Exercise 17** Push the values discussed above in order to the empty transcript. Also compute proofs for the openings of $a,b,c$ corresponding to the inputs and output. Proofs are not added to the transcript.

In [ ]:
transcript = ""
#Solve here!
value_a = #?
value_b = #?
value_c = #?
c_a = #?
c_b = #?
c_c = #?
proof_value_a       = 
proof_value_b       = 
proof_output  =

**Step 2** We now compute the challenges $\beta$ and $\gamma$ for the permutation argument from the transcript, one at a time and then push them to the transcript. With these values we can compute the $z$, $N$ and $D$ polynomials.

**Exercise 18** Compute $\beta$ from the transcript and push this value to it. Then compute $\gamma$ and similarly, push it to the transcript. Finally interpolate values for $z, N,D$ and commit $z$ to the transcript.

In [ ]:
#Solve here!
beta = #?
gamma = #?
# interpolate and commit z
z_poly, N_poly, D_poly = #?
c_z = #?


**Step 3** We are now going to build a "master polynomial" which will encode the gate constraints and the permutation constraints in one, which will be more efficient and equivalent to checking each constrain separately. 

**Exercise 19** Generate a challenge $\alpha$ and push it to the transcript.  Compute the gates constraing on the blinded polynomials:

$$ t\_gates = qM \cdot a\_blind \cdot b\_blind + qL \cdot a\_blind + qR \cdot b\_blind - c\_blind$$

Then compute the $L_1$ function as above and the constraints:

$$ t\_perm\_start = (z\_poly - 1) \cdot L_1$$

$$ t\_perm\_step = z\_poly \cdot N\_poly - D\_poly \cdot z\_poly(x \cdot \omega)$$

Compute the master polynomial:

$$ bigt = t\_gates + \alpha \cdot t\_perm\_start + \alpha^2 \cdot t\_perm\_step $$

and its quotient:

$$ quotient\_poly = \frac{bigt}{ZH}$$

Commit to $quotient\_poly$ and push to the transcript.

In [ ]:
#Solve here!
alpha = 
L1 = 
t_gates     = 
t_perm_start= 
z_shifted   = 
t_perm_step = 
bigt =  
quotient_poly = 
c_t = 


**Step 5** Now we generate the $\zeta$ challenge to open $a,b,c$ (blinded) and $quotient\_poly, z\_poly$ at $\zeta$. This will later allow the verifier to independently recompute the constraint polynomials. We also need an opening of $z\_poly$ at $\zeta \cdot \omega$.

**Exercise 20** Compute $\zeta$ from the transcript and push it. Then compute the evaluations of the blinded $a,b,c$, as well as the quotient polynomial and $z$. Compute the proofs for each of these evaluations. 

In [ ]:
zeta =
a_zeta = ;    b_zeta = ;    c_zeta = 
z_zeta = ;     t_zeta = 
z_zeta_omega = 

# individual KZG proofs
proof_a       = 
proof_b       = 
proof_c       = 
proof_z       = 
proof_t       = 
proof_z_omega = 


Finally we collect all proof artifacts we need to share with the verifier so that they can independently reconstruct the transcript and check the expected values and commitment proofs on their side.

In [37]:
# 6. Collect everything to send to the verifier
proof_dictionary = {
    'commitments': {
        'c_a': c_a,
        'c_b': c_b,
        'c_c': c_c,
        'c_z': c_z,
        'c_t': c_t
    },
    'challenges': {
        'beta': beta,
        'gamma': gamma,
        'alpha': alpha,
        'zeta': zeta
    },
    'evaluations': {
        'value_a': value_a,
        'value_b': value_b,
        'output': output,
        'a_zeta': a_zeta,
        'b_zeta': b_zeta,
        'c_zeta': c_zeta,
        'z_zeta': z_zeta,
        't_zeta': t_zeta,
        'z_zeta_omega': z_zeta_omega
    },
    'proofs': {
        'proof_value_a':proof_value_a,
        'proof_value_b':proof_value_b,
        'proof_output': proof_output,
        'proof_a': proof_a,
        'proof_b': proof_b,
        'proof_c': proof_c,
        'proof_z': proof_z,
        'proof_t': proof_t,
        'proof_z_omega': proof_z_omega
    }
}

# proof_dictionary is now ready for the verifier.

## Verifier

Now all that is left for the verifier is to reconstruct step by step the transcript and check that everything passes! If you implemented the above exercises correctly, this check should pass.

In [38]:
def verify_plonk(proof_dictionary):
    # 0. Reconstruct transcript
    transcript = ""
    if (value_a != 0 or value_b != 1):
        return False, "Input values are not correct"
    # 1. Round 1: commitments to a, b, c
    for name in ('value_a','value_b','output'):
        transcript = push(proof_dictionary['evaluations'][name], transcript)    
    for name in ('c_a','c_b','c_c'):
        transcript = push(proof_dictionary['commitments'][name], transcript)
    
    # 2. Round 2: beta, gamma
    beta_v = generate_challenge(transcript)
    if beta_v != proof_dictionary['challenges']['beta']:
        print(beta_v, proof_dictionary['challenges']['beta'])
        return False, "beta mismatch"
    transcript = push(beta_v, transcript)
    

    gamma_v = generate_challenge(transcript)
    if gamma_v != proof_dictionary['challenges']['gamma']:
        return False, "gamma mismatch"
    transcript = push(gamma_v, transcript)
    
    # append z commitment
    transcript = push(proof_dictionary['commitments']['c_z'], transcript)
    
    # 3. Round 3: alpha
    alpha_v = generate_challenge(transcript)
    if alpha_v != proof_dictionary['challenges']['alpha']:
        return False, "alpha mismatch"
    transcript = push(alpha_v, transcript)
    
    # append t commitment
    transcript = push(proof_dictionary['commitments']['c_t'], transcript)
    
    # 4. Round 4: zeta
    zeta_v = generate_challenge(transcript)
    if zeta_v != proof_dictionary['challenges']['zeta']:
        return False, "zeta mismatch"
    transcript = push(zeta_v, transcript)
    
    
    # 5. Pairing checks for each opening
    checks = [
        # inputs and output openings
        ('c_a',          'proof_value_a',         'value_a', ω),
        ('c_b',          'proof_value_b',         'value_b',ω),
        ('c_c',          'proof_output',         'output',ω^4),
        # witness openings
        ('c_a',          'proof_a',         'a_zeta', zeta_v),
        ('c_b',          'proof_b',         'b_zeta',zeta_v),
        ('c_c',          'proof_c',         'c_zeta',zeta_v),
        # z and t openings
        ('c_z',          'proof_z',         'z_zeta',zeta_v),
        ('c_t',          'proof_t',         't_zeta',zeta_v),
        # z(ζ·ω) for perm check
        ('c_z',          'proof_z_omega',   'z_zeta_omega',zeta_v* ω ),
    ]
    
    for (c_key, π_key, b_key, challenge_point) in checks:
        C = proof_dictionary['commitments'][c_key]
        π = proof_dictionary['proofs'][π_key]
        b = proof_dictionary['evaluations'][b_key]
    
        if not verification(C, π, challenge_point, b):
            return False, f"Pairing check failed for {c_key}"

    qL_z = qL(zeta_v)
    qR_z = qR(zeta_v)
    qM_z = qM(zeta_v)
    N_z = N_poly(zeta_v)
    D_z = D_poly(zeta_v)

    # Check 1: Gate constraints
    
    t_gates_v = qM_z*a_zeta*b_zeta + qL_z*a_zeta+qR_z*b_zeta-c_zeta

    # Check 2: Permutation constraints
    L1_z = L1(zeta_v)
    t_perm_start_v = (z_zeta - 1) * L1_z
    t_perm_step_v = z_zeta * N_z - D_z * z_zeta_omega

    # Check 3: Quotient constraint
    # Reconstruct the value of the master polynomial at zeta
    master_poly_v = t_gates_v + alpha_v * t_perm_start_v + alpha_v**2 * t_perm_step_v

    # Reconstruct the value of the vanishing polynomial at zeta
    ZH_z = zeta_v**n - 1

    # The final check: Does the master polynomial identity hold?
    if master_poly_v == t_zeta * ZH_z:
        print("Quotient constraint holds.")
    else:
        return False, "Quotient constraint check failed!"    

    return True, "All checks passed!"

# --- Example invocation ---
ok, msg = verify_plonk(proof_dictionary)
print("Verifier result:", ok, msg)

Quotient constraint holds.
Verifier result: True All checks passed!


## 9.Recap and conclusions

In this notebook we have introduce PLONK by means of a step by step reconstruction of the main ideas in the original paper. We have taken some liberties regarding the exact details as in the original paper for simplicity of exposition. However we have tried to stay as faithful as possible to the main ideas, while sometimes sacrificing efficiency optimization for clarity of explanation.
